# Data Aggregation and Group Operations

## There seems to be two species of Groupby: aggregators and transformers.  Aggregator reduce a section to a summary value.  Transformer change one section to another (selecting a subset of rows by some criteria is a possibility.

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## GroupBy Mechanics

In [172]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.350883,-0.036594
1,a,two,-1.796839,1.315753
2,b,one,1.094787,-0.287388
3,b,two,-1.070083,-0.615581
4,a,one,-0.217309,0.359803


In [173]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [174]:
grouped.mean()

key1
a   -0.788344
b    0.012352
Name: data1, dtype: float64

In [175]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.284096
      two    -1.796839
b     one     1.094787
      two    -1.070083
Name: data1, dtype: float64

In [176]:
means.unstack()

key2,one,two
key1,,
a,-0.284096,-1.796839
b,1.094787,-1.070083


### Ultimately, the splitting is based on an array of arbitrary labels, of the same length as the split axis.  The array can be specified directly, by a column name or a list, or by mapping as in a function call or a dictionary


In [187]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df[['data1','data2']].groupby([states, years]).mean()

data1     data2
California 2005 -1.796839  1.315753
           2006  1.094787 -0.287388
Ohio       2005 -0.710483 -0.326087
           2006 -0.217309  0.359803

In [188]:
df.groupby('key1').mean()
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.284096  0.161604
     two  -1.796839  1.315753
b    one   1.094787 -0.287388
     two  -1.070083 -0.615581

In [20]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

### each iteration returns the group key and Series or DataFrame object

In [189]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)
    print(type(group))

a
  key1 key2     data1     data2
0    a  one -0.350883 -0.036594
1    a  two -1.796839  1.315753
4    a  one -0.217309  0.359803
<class 'pandas.core.frame.DataFrame'>
b
  key1 key2     data1     data2
2    b  one  1.094787 -0.287388
3    b  two -1.070083 -0.615581
<class 'pandas.core.frame.DataFrame'>


In [23]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
4    a  one  1.965781  1.246435
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.478943  0.092908
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
('b', 'two')
  key1 key2    data1     data2
3    b  two -0.55573  0.769023


In [28]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023


In [36]:
# dtypes is a Series of types indexed by column

df.dtypes
grouped = df.groupby(df.dtypes, axis=1)

print(df.dtypes, '\n\n', df)

key1      object
key2      object
data1    float64
data2    float64
dtype: object 

   key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023
4    a  one  1.965781  1.246435


In [37]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.204708  1.393406
1  0.478943  0.092908
2 -0.519439  0.281746
3 -0.555730  0.769023
4  1.965781  1.246435
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [192]:
df.groupby(['key1', 'key2'])['data2'].mean()
# df.groupby(['key1', 'key2'])[['data2']].mean()  # if filter is a list, you will get a dataframe

key1  key2
a     one     0.161604
      two     1.315753
b     one    -0.287388
      two    -0.615581
Name: data2, dtype: float64

In [193]:
s_grouped = df.groupby(['key1', 'key2'])[[ 'data2']]
s_grouped
s_grouped.mean()

data2
key1 key2          
a    one   0.161604
     two   1.315753
b    one  -0.287388
     two  -0.615581

### Grouping with Dicts and Series

In [197]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,0.958719,-0.838950,1.150807,0.335063,-1.440063
Steve,1.007140,-0.148027,-0.840961,-0.470617,1.865265
Wes,-0.498365,NaN,NaN,0.669574,0.105577
Jim,-0.332262,1.780158,-0.018843,-0.786404,-0.858600
Travis,0.818913,0.423509,-0.331155,-0.336931,-0.957532


In [198]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [199]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,1.485870,-1.320293
Steve,-1.311578,2.724377
Wes,0.669574,-0.392787
Jim,-0.805247,0.589296
Travis,-0.668086,0.284890


In [201]:
# mapping dict internally converted to array

by_column = people.groupby(['red','red','blue', 'blue', 'red'], axis=1)
by_column.sum()

,blue,red
Joe,1.485870,-1.320293
Steve,-1.311578,2.724377
Wes,0.669574,-0.392787
Jim,-0.805247,0.589296
Travis,-0.668086,0.284890


In [202]:
map_series = pd.Series(mapping)
map_series
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions

In [206]:
def vowel_or_consonant(c):
    if c in "aeiou":
        return 'vowel'
    else:
        return 'consonant'
    
# each element of the axis is passed to the function    
people.groupby(vowel_or_consonant, axis=1).sum()   

people.groupby(len).sum()

,a,b,c,d,e
3,0.128092,0.941208,1.131964,0.218234,-2.193085
5,1.007140,-0.148027,-0.840961,-0.470617,1.865265
6,0.818913,0.423509,-0.331155,-0.336931,-0.957532


In [207]:
# mix and match filter formats is fine
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.498365 -0.838950  1.150807  0.335063 -1.440063
  two -0.332262  1.780158 -0.018843 -0.786404 -0.858600
5 one  1.007140 -0.148027 -0.840961 -0.470617  1.865265
6 two  0.818913  0.423509 -0.331155 -0.336931 -0.957532

### Grouping by Index Levels

In [215]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.793555 -1.169936 -0.560818  0.568244  0.361082
1      0.942275 -1.888889 -0.619714 -0.095563 -2.774391
2      0.452920  2.739776 -0.136435 -2.393160  0.664261
3     -1.557638 -0.069172  0.331131  0.920039  1.086511

In [217]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [213]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=[1, 3, 5, 1, 3])
hier_df.groupby(['US', 'US', 'US', 'JP', 'JP'], axis=1).count()

,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## Data Aggregation

### Aggregation produce scalar values from arrays

In [220]:
df
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

#grouped = df['data1'].groupby(df['key1'])
#grouped.quantile(0.9)

key1
a   -0.244024
b    0.878300
Name: data1, dtype: float64

In [228]:
grouped = df.drop('key2', axis=1).groupby('key1')
for label, group in grouped:
    print(label)
    print(group)

a
  key1     data1     data2
0    a -0.350883 -0.036594
1    a -1.796839  1.315753
4    a -0.217309  0.359803
b
  key1     data1     data2
2    b  1.094787 -0.287388
3    b -1.070083 -0.615581


In [229]:
def peak_to_peak(arr):
    print("Hello")
    print(arr)
    print("Returns", arr.max() - arr.min())  # must throw exception where not supported, eliminating column
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

Hello
0   -0.350883
1   -1.796839
4   -0.217309
Name: data1, dtype: float64
Returns 1.5795300240680883
Hello
2    1.094787
3   -1.070083
Name: data1, dtype: float64
Returns 2.1648698665913235
Hello
0   -0.036594
1    1.315753
4    0.359803
Name: data2, dtype: float64
Returns 1.3523474058626228
Hello
2   -0.287388
3   -0.615581
Name: data2, dtype: float64
Returns 0.32819249979498377


,data1,data2
key1,,
a,1.57953,1.352347
b,2.16487,0.328192


In [230]:
df

,key1,key2,data1,data2
0,a,one,-0.350883,-0.036594
1,a,two,-1.796839,1.315753
2,b,one,1.094787,-0.287388
3,b,two,-1.070083,-0.615581
4,a,one,-0.217309,0.359803


In [231]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.788344  0.875932 -1.796839 -1.073861 -0.350883 -0.284096   
b      2.0  0.012352  1.530794 -1.070083 -0.528865  0.012352  0.553570   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a    -0.217309   3.0  0.546320  0.695200 -0.036594  0.161604  0.359803   
b     1.094787   2.0 -0.451484  0.232067 -0.615581 -0.533533 -0.451484   

                          
           75%       max  
key1                      
a     0.837778  1.315753  
b    -0.369436 -0.287388

### Column-Wise and Multiple Function Application

In [232]:
tips = pd.read_csv('examples/tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [233]:
grouped = tips.groupby(['day', 'smoker'])

In [234]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg([('mean label', 'mean')])  # give it a label
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [235]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

Hello
91     0.155625
94     0.142857
99     0.120385
223    0.187735
Name: tip_pct, dtype: float64
Returns 0.06734943559063597
Hello
90     0.103555
92     0.173913
93     0.263480
95     0.117750
96     0.146628
97     0.124688
98     0.142789
100    0.220264
101    0.195059
220    0.180921
221    0.259314
222    0.223776
224    0.117735
225    0.153657
226    0.198216
Name: tip_pct, dtype: float64
Returns 0.15992499001671778
Hello
19     0.162228
20     0.227679
21     0.135535
22     0.141408
23     0.192288
24     0.160444
25     0.131387
26     0.149589
27     0.157604
28     0.198157
29     0.152672
30     0.151832
31     0.136240
32     0.199203
33     0.118415
34     0.183915
35     0.149626
36     0.122624
37     0.181335
38     0.123596
39     0.159898
40     0.139651
57     0.056797
59     0.139424
64     0.150085
65     0.156873
66     0.150152
68     0.099357
70     0.163894
71     0.175747
74     0.149355
75     0.118934
104    0.195029
108    0.206140
110    0.214286
11

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [73]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [76]:
functions = ['count', 'mean', 'max']
result = grouped[[ 'tip_pct', 'total_bill']].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [77]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [79]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [242]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
            'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [243]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## Apply: General split-apply-combine

In [248]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column, ascending=False)[:n]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


## For apply it is a dataframe in and a dataframe out



In [249]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [250]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [251]:
result = tips.groupby('smoker')['tip_pct'].describe()
result
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

f = lambda x: x.describe()
grouped.apply(f)

### Suppressing the Group Keys

In [252]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


### Quantile and Bucket Analysis

In [253]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-1.705, -0.101]
1    (-1.705, -0.101]
2    (-3.316, -1.705]
3    (-1.705, -0.101]
4    (-1.705, -0.101]
5     (-0.101, 1.504]
6    (-1.705, -0.101]
7     (-0.101, 1.504]
8      (1.504, 3.109]
9     (-0.101, 1.504]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.316, -1.705] < (-1.705, -0.101] < (-0.101, 1.504] < (1.504, 3.109]]

In [280]:
# This function takes a Series of values ()

def get_stats(group):
#    print(group.head())
    return pd.Series({'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()})
grouped = frame.data1.groupby(quartiles)

grouped.apply(get_stats).unstack()

#frame.apply(get_stats)

,min,max,count,mean
data1,,,,
"(-3.316, -1.705]",-3.309894,-1.709252,28.0,-2.304473
"(-1.705, -0.101]",-1.698828,-0.103850,426.0,-0.727669
"(-0.101, 1.504]",-0.098828,1.503080,474.0,0.559730
"(1.504, 3.109]",1.517647,3.108665,72.0,1.878822


In [283]:
# function gets passed one Series per column (across axis 0), returns a wholly newly formed Series)
# transform is said to be preferred for performance.
# transform requires the index to stay intact and can call numba or Cython engines.
frame.apply(get_stats, axis=0)

,data1,data2
min,-3.309894,-2.810658
max,3.108665,3.403968
count,1000.000000,1000.000000
mean,0.026075,0.048925


### Example: Filling Missing Values with Group-Specific       Values

In [284]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s
s.fillna(s.mean())

0    0.295167
1    0.149627
2    0.295167
3   -0.469871
4    0.295167
5    1.205743
dtype: float64

In [293]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states, name='snurk')
data

Ohio          0.912859
New York     -1.113540
Vermont       0.265249
Florida      -0.844568
Oregon       -0.656408
Nevada        1.102397
California    0.028681
Idaho         0.208205
Name: snurk, dtype: float64

In [294]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data
data.groupby(group_key).mean()

East   -0.348416
West   -0.313863
Name: snurk, dtype: float64

In [295]:
fill_mean = lambda g: g.fillna(g.mean())
#data.groupby(group_key).transform(fill_mean)
data.groupby(group_key).apply(fill_mean)

Ohio          0.912859
New York     -1.113540
Vermont      -0.348416
Florida      -0.844568
Oregon       -0.656408
Nevada       -0.313863
California    0.028681
Idaho        -0.313863
Name: snurk, dtype: float64

In [296]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.912859
New York     -1.113540
Vermont       0.500000
Florida      -0.844568
Oregon       -0.656408
Nevada       -1.000000
California    0.028681
Idaho        -1.000000
Name: snurk, dtype: float64

In [297]:
data.name

'snurk'

### Example: Random Sampling and Permutation

In [298]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'Q', 'K']
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [299]:
deck[:15]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
AS      1
2S      2
dtype: int64

In [300]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

AS      1
AC      1
2S      2
10D    10
2D      2
dtype: int64

In [303]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

MultiIndex([('C',  '2C'),
            ('C',  '3C'),
            ('D', '10D'),
            ('D',  '8D'),
            ('H',  'JH'),
            ('H',  '6H'),
            ('S',  '6S'),
            ('S',  '3S')],
           )

In [306]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

5C     5
QC    10
5D     5
9D     9
2H     2
JH    10
AS     1
3S     3
dtype: int64

### Example: Group Weighted Average and Correlation

In [307]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,0.979395,0.191258
1,a,0.250284,0.539750
2,a,-0.205707,0.880692
3,a,0.818932,0.170234
4,b,-1.092945,0.153200
5,b,1.433864,0.876550
6,b,-0.349092,0.031772
7,b,0.448873,0.108232


In [309]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.15750
b    0.96337
dtype: float64

In [310]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True,
                       index_col=0)
close_px.info()
close_px[-4:]

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [311]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [312]:
close_px.pct_change().head()

,AAPL,MSFT,XOM,SPX
2003-01-02,NaN,NaN,NaN,NaN
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086


In [313]:
rets = close_px.pct_change().dropna()
rets.index

DatetimeIndex(['2003-01-03', '2003-01-06', '2003-01-07', '2003-01-08',
               '2003-01-09', '2003-01-10', '2003-01-13', '2003-01-14',
               '2003-01-15', '2003-01-16',
               ...
               '2011-10-03', '2011-10-04', '2011-10-05', '2011-10-06',
               '2011-10-07', '2011-10-10', '2011-10-11', '2011-10-12',
               '2011-10-13', '2011-10-14'],
              dtype='datetime64[ns]', length=2213, freq=None)

In [315]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [326]:
a= by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))
a.name='abc'
a.index.name='year'
pd.DataFrame(a)

,abc
year,
2003,0.480868
2004,0.259024
2005,0.300093
2006,0.161735
2007,0.417738
2008,0.611901
2009,0.432738
2010,0.571946
2011,0.581987


### Example: Group-Wise Linear Regression

In [136]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [139]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## Pivot Tables and Cross-Tabulation

In [327]:
tips.columns

Index(['total_bill', 'tip', 'smoker', 'day', 'time', 'size', 'tip_pct'], dtype='object')

In [143]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [148]:
tips.groupby(['day','smoker']).mean().reindex(columns=['size','tip','tip_pct','total_bill'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [155]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day', 'smoker'])


size   tip_pct
time   day  smoker                    
Dinner Fri  No      2.000000  0.139622
            Yes     2.222222  0.165347
       Sat  No      2.555556  0.158048
            Yes     2.476190  0.147906
       Sun  No      2.929825  0.160113
            Yes     2.578947  0.187250
       Thur No      2.000000  0.159744
Lunch  Fri  No      3.000000  0.187735
            Yes     1.833333  0.188937
       Thur No      2.500000  0.160311
            Yes     2.352941  0.163863

In [154]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day', 'smoker']).unstack()


size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [151]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [156]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [160]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc='count', margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [161]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
...                      ...       ...       ...       ...
Lunch  3    Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

[21 rows x 4 columns]

### Cross-Tabulations: Crosstab

### This is just a special case of pivot for counting classes for a particular combination of rows and columns.  No numerical aggregations.  Margins is supported.

In [162]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [163]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [164]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [165]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [167]:
pd.crosstab([tips.time], [tips.smoker, tips.day], margins=True)

smoker  No              Yes               All
day    Fri Sat Sun Thur Fri Sat Sun Thur     
time                                         
Dinner   3  45  57    1   9  42  19    0  176
Lunch    1   0   0   44   6   0   0   17   68
All      4  45  57   45  15  42  19   17  244

In [166]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

## Conclusion